# Remapping analysis with pairs of neurons during search and homing part of the trials

We get the IFR and the firing rate maps for the neurons in different condition. We then look at the IFR association and firing rate map similarity for pairs of neurons in a given condition.

* Do cells that fire together in one environment also fire together in the next environment?
* Do cells with similar firing rate maps in one condition also have similar maps in another?


In [3]:
%load_ext autoreload
%autoreload 2
%run setup_project.py
prepareSessionsForSpatialAnalysisProject(sSesList,myProject.sessionList,pose_file_extension = ".pose_kf.npy")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Project name: autopi_ca1
dataPath: /adata/projects/autopi_ca1
dlcModelPath: /adata/models
Reading /adata/projects/autopi_ca1/sessionList
We have 39 testing sessions in the list
See myProject and sSesList objects
Loading Animal_pose and Spike_train, sSes.ap and sSes.cg


100%|███████████████████████████████████████████| 39/39 [00:22<00:00,  1.73it/s]

Loading ses.trial_table_simple as ses.trials
Create condition intervals in ses.intervalDict


In [4]:
for ses, sSes in tqdm(zip(myProject.sessionList,sSesList)):
    getSearchHomingIntervals(ses,sSes)

39it [00:00, 55.00it/s]


## Instantaneous Firing rate associations

We can look at pairs of cells and test if their firing associations (r values) are preserve or change across different conditions.
The function below calculates the firing associations of pairs of neurons in different conditions.
Later on we can compare IFR association across conditions.

In [5]:
from scipy.stats import pearsonr
def ifrAssociationsPerCondition(ses,sSes,
                                 conditions=['searchToLeverPath_light','homingFromLeavingLever_light',
                                             'searchToLeverPath_dark','homingFromLeavingLever_dark',
                                            'searchToLeverPath_light_1','searchToLeverPath_light_2',
                                             'searchToLeverPath_dark_1','searchToLeverPath_dark_2',
                                            'homingFromLeavingLever_light_1','homingFromLeavingLever_light_2',
                                             'homingFromLeavingLever_dark_1','homingFromLeavingLever_dark_2'],
                                prefix="ifrAsso_"):
    """
    Calculate the instantaneous firing rate associations of pairs of neurons in a sets of conditions
    
    Arguments: 
    ses: autopipy session
    sSes: spikeA session
    conditions: list of condition names you want to analyze
    prefix: prefix for the DataFrame column names
    
    Return:
    Pandas DataFrame with the IFR association for each condition. One row per pair
    """ 
    
    
    # we will store the results in a dictionary, one list of scores per condition
    res={}
    for cond in conditions:
        
        
        nInfo = len(cond.split("_"))
        if nInfo == 2:
            navPathType = cond.split("_")[0]
            light = cond.split("_")[1]
        if nInfo == 3:
            navPathType = cond.split("_")[0]
            light = cond.split("_")[1]
            light = light + "_" + cond.split("_")[2]
        
        inter = ses.intervalDict[navPathType+"_"+light]
        
        
        # set the intervals for each neuron and calculate the IFR
        for n in sSes.cg.neuron_list:
            n.spike_train.unset_intervals()
            n.spike_train.set_intervals(inter)
            n.spike_train.instantaneous_firing_rate(bin_size_sec = 0.100, sigma = 1,outside_interval_solution="remove")
        # calculate firing association for all pairs of neurons
        sSes.cg.make_pairs(pair_type="combinations")
        rs = np.empty(len(sSes.cg.pairs)) # to store the results
        for i, (j,k) in enumerate(sSes.cg.pairs):
            n1=sSes.cg.neuron_list[j]
            n2=sSes.cg.neuron_list[k]
            rs[i] = pearsonr(n1.spike_train.ifr[0].flatten(),n2.spike_train.ifr[0].flatten())[0]
        res[cond]=rs
            
    #create a DataFrame from the dictionary
    res = pd.DataFrame(res)
    res = res.add_prefix(prefix)
    res["id1"] = [ sSes.name+"_"+sSes.cg.neuron_list[i].name for i,j in sSes.cg.pairs]
    res["id2"] = [ sSes.name+"_"+sSes.cg.neuron_list[j].name for i,j in sSes.cg.pairs]
    
    res["session"] = sSes.name
    return res


In [6]:
ses,sSes = list(zip(myProject.sessionList,sSesList))[0]
res = ifrAssociationsPerCondition(ses,sSes)

/home/kevin/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [7]:
res

,ifrAsso_searchToLeverPath_light,ifrAsso_homingFromLeavingLever_light,ifrAsso_searchToLeverPath_dark,ifrAsso_homingFromLeavingLever_dark,ifrAsso_searchToLeverPath_light_1,ifrAsso_searchToLeverPath_light_2,ifrAsso_searchToLeverPath_dark_1,ifrAsso_searchToLeverPath_dark_2,ifrAsso_homingFromLeavingLever_light_1,ifrAsso_homingFromLeavingLever_light_2,ifrAsso_homingFromLeavingLever_dark_1,ifrAsso_homingFromLeavingLever_dark_2,id1,id2,session
0,-0.093087,-0.054715,0.110710,0.074444,-0.102040,-0.099412,0.090788,0.125072,-0.002338,-0.105723,0.120710,0.032899,mn5824-20112020-0107_80,mn5824-20112020-0107_90,mn5824-20112020-0107
1,-0.051358,-0.047985,-0.160393,0.198994,-0.044447,-0.023785,-0.139141,-0.172280,-0.058228,-0.046501,0.155394,0.235794,mn5824-20112020-0107_80,mn5824-20112020-0107_92,mn5824-20112020-0107
2,-0.069240,-0.069421,-0.002329,0.147623,-0.059115,-0.078931,0.084147,-0.046974,-0.138440,-0.014876,0.207908,0.092063,mn5824-20112020-0107_80,mn5824-20112020-0107_96,mn5824-20112020-0107
3,0.011641,0.079067,-0.085742,-0.093057,0.052123,-0.008109,-0.232324,0.039475,0.197515,-0.074400,-0.054414,-0.112308,mn5824-20112020-0107_80,mn5824-20112020-0107_98,mn5824-20112020-0107
4,0.002969,-0.080904,-0.113453,-0.161225,0.086735,-0.109818,-0.111415,-0.121045,-0.073806,-0.080537,-0.205182,-0.121958,mn5824-20112020-0107_80,mn5824-20112020-0107_100,mn5824-20112020-0107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,0.005455,-0.060056,-0.018187,-0.031248,0.025404,-0.032039,-0.071635,0.050507,-0.039475,-0.075610,-0.028219,-0.034821,mn5824-20112020-0107_116,mn5824-20112020-0107_122,mn5824-20112020-0107
62,-0.006533,0.049023,-0.018390,-0.023857,NaN,-0.005658,NaN,-0.023709,0.068935,NaN,-0.018207,-0.029199,mn5824-20112020-0107_116,mn5824-20112020-0107_124,mn5824-20112020-0107
63,-0.043810,0.037088,-0.005964,-0.024279,-0.017989,-0.065889,0.037551,-0.054092,-0.015224,0.119469,0.049225,-0.046064,mn5824-20112020-0107_118,mn5824-20112020-0107_122,mn5824-20112020-0107
64,-0.006839,-0.012235,-0.012957,-0.025037,NaN,-0.011637,NaN,-0.017454,-0.017847,NaN,-0.022929,-0.025629,mn5824-20112020-0107_118,mn5824-20112020-0107_124,mn5824-20112020-0107


In [8]:
res_ifr=pd.concat([ifrAssociationsPerCondition(ses,sSes) for ses,sSes in tqdm(zip(myProject.sessionList,sSesList))],ignore_index=True)

0it [00:00, ?it/s]/home/kevin/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
39it [07:23, 11.38s/it]


## Firing rate map similarities

As a complementary analysis to the IFR association, we can also look at firing rate map similarity in different conditions. 


In [9]:
a = 'searchToLeverPath_light_1'
a.split("_")

['searchToLeverPath', 'light', '1']

In [10]:
from scipy.stats import pearsonr
def map_cor(a,b):
    """
    Correlation coefficient between two firing rate maps
    
    Arguments:
    a: 2D np.array (map1)
    b: 2D np.array (map2)
    
    Returns:
    Pearson correlation coefficient between a and b
    """
    a = a.flatten()
    b = b.flatten()
    indices = np.logical_and(~np.isnan(a), ~np.isnan(b))
    r,p = pearsonr(a[indices],b[indices])
    return r

def mapSimilarityPerCondition(ses,sSes, onlyArena=True,
                              conditions=['searchToLeverPath_light','homingFromLeavingLever_light','searchToLeverPath_dark','homingFromLeavingLever_dark',
                                          'searchToLeverPath_light_1', 'searchToLeverPath_light_2', 'searchToLeverPath_dark_1', 'searchToLeverPath_dark_2',
                                          'homingFromLeavingLever_light_1', 'homingFromLeavingLever_light_2', 'homingFromLeavingLever_dark_1', 'homingFromLeavingLever_dark_2'],
                              prefix="mapSim_"):
    """
    Calculate the map similarity of pairs of neurons in a sets of conditions
    
    Arguments: 
    ses: autopipy session
    sSes: spikeA session
    conditions: list of condition names you want to analyze
    prefix: prefix for the DataFrame column names
    
    Return:
    Pandas DataFrame with map similarity for each condition. One row per pair
    """ 
    # we will store the results in a dictionary, one list of scores per condition
    
    
    res={}
    xy_range=np.array([[-50,-90],[50,60]])
    
    for cond in conditions:
        # set the intervals for each neuron and animal pose and calculate the maps
        
        nInfo = len(cond.split("_"))
        if nInfo == 2:
            navPathType = cond.split("_")[0]
            light = cond.split("_")[1]
        if nInfo == 3:
            navPathType = cond.split("_")[0]
            light = cond.split("_")[1]
            light = light + "_" + cond.split("_")[2]
            
            
        inter = ses.intervalDict[navPathType+"_"+light]
        
        
        sSes.ap.set_intervals(inter)
       
        if onlyArena:
            # this should come after setting the time intervals, will be reset when calling sSes.ap.set_intervals again
            sSes.ap.invalid_outside_spatial_area(shape="circle",radius=43.0,center=np.array([0,0])) ## remove the bridge and homebase

        
        
        for n in sSes.cg.neuron_list:
            n.spike_train.set_intervals(inter)
            n.spatial_properties.firing_rate_map_2d(cm_per_bin =3, smoothing_sigma_cm = 3, smoothing=True,xy_range=xy_range)
            
            
        # calculate map similarity for all pairs of neurons
        sSes.cg.make_pairs(pair_type="combinations")
        rs = np.empty(len(sSes.cg.pairs)) # to store the results
        for i, (j,k) in enumerate(sSes.cg.pairs):
            n1=sSes.cg.neuron_list[j]
            n2=sSes.cg.neuron_list[k]
            rs[i] = map_cor(n1.spatial_properties.firing_rate_map,n2.spatial_properties.firing_rate_map)
        res[cond]=rs
            
    #create a DataFrame from the dictionary
    res = pd.DataFrame(res)
    res = res.add_prefix(prefix)
    res["id1"] = [ sSes.name+"_"+sSes.cg.neuron_list[i].name for i,j in sSes.cg.pairs]
    res["id2"] = [ sSes.name+"_"+sSes.cg.neuron_list[j].name for i,j in sSes.cg.pairs]
    
    res["session"] = sSes.name
    return res

In [11]:
ses,sSes = list(zip(myProject.sessionList,sSesList))[0]
res = mapSimilarityPerCondition(ses,sSes)

In [12]:
res_map=pd.concat([mapSimilarityPerCondition(ses,sSes) for ses,sSes in tqdm(zip(myProject.sessionList,sSesList))],ignore_index=True)

39it [06:48, 10.49s/it]


In [13]:
res_map.shape

(29841, 15)

## Check data integrity of ifr and mapSim df and merge

In [14]:
print(res_map.shape,res_ifr.shape)
if res_map.shape != res_ifr.shape:
    print("problem with the shape of the results")
else:
    print("shape of 2 dfs is matching")

(29841, 15) (29841, 15)
shape of 2 dfs is matching


# Save the data

In [15]:
res = pd.concat([res_ifr,res_map.iloc[:,0:12]],axis=1)
fn=myProject.dataPath+"/results/searchHomingPairs_ifrAsso_mapSim.csv"
print("Saving to",fn)
res.to_csv(fn,index=False)

Saving to /adata/projects/autopi_ca1/results/searchHomingPairs_ifrAsso_mapSim.csv
